# Get the data

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np

from sklearn import linear_model
from sklearn import svm
from skimage.feature import hog
from sklearn.metrics import accuracy_score, roc_curve, auc
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow.keras import datasets
import matplotlib.pyplot as plt

(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()
train_images.shape

(50000, 32, 32, 3)

# Prepare the 5 fold X-validation split

In [2]:
def get_x_valid_data(train_x, train_y, test_x, test_y, folds=5):
    # flatten
    flat_train_x = train_x.reshape((len(train_x), 32*32*3))
    flat_train_y = train_y.reshape((len(train_y), 1))
    flat_test_x = test_x.reshape((len(test_x), 32*32*3))
    flat_test_y = test_y.reshape((len(test_y), 1))
    
    # combine
    all_x = np.vstack((flat_train_x, flat_test_x))
    all_y = np.vstack((flat_train_y, flat_test_y))
    
    # scale 
    ss = StandardScaler()
    all_x = ss.fit_transform(all_x)
    
    # create X-Validation Split
    xvalid_train_X = []
    xvalid_train_Y = []
    xvalid_test_X = []
    xvalid_test_Y = []
    width = (int)(all_x.shape[0] / folds)
    for i in range(folds):        
        xvalid_train_X.append(np.vstack((all_x[:i*width],all_x[(i+1)*width:])))
        xvalid_train_Y.append(np.vstack((all_y[:i*width],all_y[(i+1)*width:])))
        xvalid_test_X.append(all_x[i*width:(i+1)*width])
        xvalid_test_Y.append(all_y[i*width:(i+1)*width])
    xvalid_train_X = np.array(xvalid_train_X)
    xvalid_train_Y = np.array(xvalid_train_Y)
    xvalid_test_X = np.array(xvalid_test_X)
    xvalid_test_Y = np.array(xvalid_test_Y)
    print(xvalid_train_X.shape)
    print(xvalid_train_Y.shape)
    print(xvalid_test_X.shape)
    print(xvalid_test_Y.shape)
    return xvalid_train_X, xvalid_train_Y, xvalid_test_X, xvalid_test_Y

xvalid_train_X, xvalid_train_Y, xvalid_test_X, xvalid_test_Y = get_x_valid_data(train_images, train_labels, test_images, test_labels)

c:\python\lib\site-packages\sklearn\utils\validation.py:590: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
c:\python\lib\site-packages\sklearn\utils\validation.py:590: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


(5, 48000, 3072)
(5, 48000, 1)
(5, 12000, 3072)
(5, 12000, 1)


# Train 5 different SVD:
- **Make sure you change the 'n_jobs' parameter to reflect the \# of cores in your system**
- Outputs each splits AUC value and the Accuracy rate. Also, calculates the average of the values:
- You can also turn off 'verbose' parameter to get rid of the unnecessary outputs. You just won't be able to see the progress until the entire process is finished.

In [20]:
folds = 5
auc_list = []
acc_list = []
# fit models and get results
for i in range(folds):
    sgdClassifierAll = linear_model.SGDClassifier(loss="hinge", penalty='l2',
                    alpha=0.0001, l1_ratio=0.15, fit_intercept=True,
                    max_iter=1000, tol=1e-4, shuffle=True, verbose=1,
                    epsilon=0.1, n_jobs=4, random_state=1010, learning_rate='optimal',
                    eta0=0.0, power_t=0.5, early_stopping=False, validation_fraction=0.1,
                    n_iter_no_change=10, class_weight=None, warm_start=False,
                    average=False)
    sgdClassifierAll.fit(xvalid_train_X[i], xvalid_train_Y[i])
    predictions = sgdClassifierAll.predict(xvalid_test_X[i])
    
    # calculate validation metrics
    a_sum = 0
    for j in range(10): # every class
        fpr, tpr, thresholds = roc_curve(xvalid_test_Y[i], predictions, pos_label=i)
        a_sum += auc(fpr, tpr)
    a_average = a_sum / 10
    b = accuracy_score(xvalid_test_Y[i], predictions)
    auc_list.append(a_average)
    acc_list.append(b)
    

# print results
for i in range(folds):
    print("{:d} - AUC: {:.2f}\tACC: {:.2f}".format(i, auc_list[i], acc_list[i]))
print("================================")
mean_auc = sum(auc_list) / folds
mean_acc = sum(acc_list) / folds
print("    AUC: {:.2f}\tACC: {:.2f}".format(mean_auc, mean_acc))

c:\python\lib\site-packages\sklearn\utils\validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.


-- Epoch 1-- Epoch 1-- Epoch 1
-- Epoch 1


Norm: 383.75, NNZs: 3072, Bias: -3171.468027, T: 48000, Avg. loss: 333.001978
Total training time: 0.65 seconds.
-- Epoch 2
Norm: 419.19, NNZs: 3072, Bias: -2547.048428, T: 48000, Avg. loss: 351.570570
Total training time: 0.65 seconds.
-- Epoch 2
Norm: 497.46, NNZs: 3072, Bias: -3139.069493, T: 48000, Avg. loss: 313.716528
Total training time: 0.74 seconds.
-- Epoch 2
Norm: 377.13, NNZs: 3072, Bias: -2674.653737, T: 48000, Avg. loss: 371.091453
Total training time: 0.78 seconds.
-- Epoch 2
Norm: 343.83, NNZs: 3072, Bias: -2863.003949, T: 96000, Avg. loss: 149.495107
Total training time: 1.13 seconds.
Convergence after 2 epochs took 1.13 seconds
-- Epoch 1
Norm: 374.38, NNZs: 3072, Bias: -2102.995665, T: 96000, Avg. loss: 159.298309
Total training time: 1.15 seconds.
Convergence after 2 epochs took 1.15 seconds
-- Epoch 1
Norm: 439.47, NNZs: 3072, Bias: -2827.333447, T: 96000, Avg. loss: 142.259803
Total training time: 1.16 seconds.
Convergen

[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    2.6s finished


{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}


c:\python\lib\site-packages\sklearn\utils\validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.


...

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}


c:\python\lib\site-packages\sklearn\utils\validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.


-- Epoch 1-- Epoch 1

-- Epoch 1
-- Epoch 1
Norm: 384.61, NNZs: 3072, Bias: -3059.017051, T: 48000, Avg. loss: 326.786687
Total training time: 0.45 seconds.
-- Epoch 2
Norm: 427.96, NNZs: 3072, Bias: -2625.631646, T: 48000, Avg. loss: 366.191125
Total training time: 0.44 seconds.
-- Epoch 2
Norm: 376.07, NNZs: 3072, Bias: -2742.414743, T: 48000, Avg. loss: 370.284388
Total training time: 0.46 seconds.
-- Epoch 2
Norm: 500.65, NNZs: 3072, Bias: -3134.364848, T: 48000, Avg. loss: 320.834704
Total training time: 0.46 seconds.
-- Epoch 2
Norm: 349.14, NNZs: 3072, Bias: -2750.904846, T: 96000, Avg. loss: 145.227474
Total training time: 0.83 seconds.
Convergence after 2 epochs took 0.83 seconds
-- Epoch 1
Norm: 381.23, NNZs: 3072, Bias: -2187.088570, T: 96000, Avg. loss: 163.753601
Total training time: 0.84 seconds.
Convergence after 2 epochs took 0.84 seconds
-- Epoch 1
Norm: 335.04, NNZs: 3072, Bias: -2321.415372, T: 96000, Avg. loss: 172.759128
Total training time: 0.86 seconds.
Convergen

[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    2.2s finished


{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}
0 - AUC: 0.31	ACC: 0.39
1 - AUC: 0.42	ACC: 0.37
2 - AUC: 0.41	ACC: 0.38
3 - AUC: 0.47	ACC: 0.39
4 - AUC: 0.47	ACC: 0.39
    AUC: 0.42	ACC: 0.38
